In [ ]:
from custom_nodes.flourence.nodes import DownloadAndLoadFlorence2Model, Florence2Run
download_and_load_florence2_model = DownloadAndLoadFlorence2Model()
florence2_model = download_and_load_florence2_model.loadmodel("microsoft/Florence-2-large", "fp16", "sdpa")

flourence_run = Florence2Run()
image, mask, caption, data = flourence_run.encode(image = None, text_input = "", florence2_model = florence2_model, task = "caption", fill_mask = True, keep_model_loaded=False, num_beams=3, max_new_tokens=1024, do_sample=True, output_mask_select="", seed=None)


In [ ]:
from custom_nodes.EasyUse.py.logic import showAnything
showAny =  showAnything()
output_of_show_anything = showAny.log_input(anything=[image, caption, data])


In [ ]:
from custom_nodes.EasyUse.py.easyNodes import positivePrompt
positivePrompt = positivePrompt()
positivePrompt_result = positivePrompt.positive_prompt("industrial interior, city skyline visible in the background")


In [ ]:
from custom_nodes.EasyUse.py.easyNodes import positivePrompt
positivePrompt2 = positivePrompt()
positivePrompt_result2 = positivePrompt2.positive_prompt("Convert the user prompt into a very concise positive prompt for creating a photographic image using SDXL. Mention important elements like style, textures, materials, colors, and other elements likelighting conditions and any other important and relvant decor elements. Start your prompt by mentioning the style of design, textures, colors and decor elements. Describe the background of the scene carefully. Use natural language to describe your prompt output. You have to take into account the user input to create your designs. Don't mention the words 3D rendering, hand drawn, or sketch in your prompt. Make your designs aesthetically pleasing. Stick closely to structure of building in the input for composition. If it's a building and not a room, mention the bakgound setting also. Make your designs aesthetically pleasing.For both cases, start your prompt by writing: Architecture photography of a")


In [ ]:
from custom_nodes.wasnodesuite.WAS_Node_Suite import WAS_Search_and_Replace, WAS_Text_Concatenate
was_search_and_replace = WAS_Search_and_Replace()
reuslt_text,  = was_search_and_replace.text_search_and_replace(caption, "The image", ".")

was_text_concatenate = WAS_Text_Concatenate()
result_text = was_text_concatenate.text_concatenate(delimiter= "User input is:", clean_whitespace= "true", kwargs={"text_a": reuslt_text, "text_b": positivePrompt_result})

was_text_concatenate2 = WAS_Text_Concatenate()
result_text2 = was_text_concatenate2.text_concatenate(delimiter= ",", clean_whitespace= "true", kwargs={"text_a": positivePrompt2, "text_b": result_text})


In [ ]:
from nodes import LoadImage
load_image = LoadImage()
# sexy
image, = load_image.load_image("https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png")


In [ ]:
from custom_nodes.LayerStyle.py.image_scale_by_aspect_ratio_v2 import ImageScaleByAspectRatioV2
image_scale_by_aspect_ratio_v2 = ImageScaleByAspectRatioV2()
image, = image_scale_by_aspect_ratio_v2.image_scale_by_aspect_ratio(image, aspect_ratio = "original", proportional_width = 1, proportional_height = 1, fit = "crop", method = "lanczos", round_to_multiple = 64, scale_to_side = "shortest", scale_to_length = 832, background_color = "#000000")

In [ ]:
from nodes import CheckpointLoaderSimple
checkpoint_loader_simple = CheckpointLoaderSimple()
# sexy
model, clip, vae = checkpoint_loader_simple.load_checkpoint("dynavision.safetensors")

In [ ]:
from nodes import VAEEncode
vae_encode = VAEEncode()
latent = vae_encode.encode(pixels=image, vae=vae)

In [ ]:
from custom_nodes.masquerade.MaskNodes import GetImageSize
get_image_size = GetImageSize()
width, height = get_image_size.get_size(image)

In [ ]:
from custom_nodes.controlnetAux import AIO_Preprocessor
aio_preprocessor = AIO_Preprocessor()
aio_image, = aio_preprocessor.execute(preprocessor="Scribble_XDoG_Preprocessor", image=image, resolution=832)

In [ ]:
from custom_nodes.artVenture.modules.llm.chat import OpenAIApiNode, LLMApiConfigNode, LLMCompletionNode
openai_api_node = OpenAIApiNode()
llm_api, = openai_api_node.create_api(openai_api_key="sk-proj-hsxbnHj3IQTUph3C3sQIT3BlbkFJthfRvT3FVGFJ03rTBBpy", endpoint="https://api.openai.com/v1")

llm_config_node = LLMApiConfigNode()
llm_config, = llm_config_node.make_config(model="gpt-4o-mini", max_token=1024, temperature=1)

llm_completion_node = LLMCompletionNode()
response, = llm_completion_node.chat(api=llm_api, config=llm_config, seed=719131606733690)

In [ ]:
from custom_nodes.EasyUse.py.logic import showAnything
showAny2 =  showAnything()
output_of_show_anything2 = showAny2.log_input(anything=[response])

In [ ]:
from custom_nodes.advancedControlNet.adv_control.nodes_main import ControlNetLoaderAdvanced
controlnet_loader_advanced = ControlNetLoaderAdvanced()
controlnet, = controlnet_loader_advanced.load_controlnet(cnet="pytorch_model_promax.safetensors")

In [ ]:
from comfy_extras.nodes_controlnet import SetUnionControlNetType
set_union_controlnet_type = SetUnionControlNetType()
controlnet, = set_union_controlnet_type.set_controlnet_type(control_net=controlnet, type="depth")

In [ ]:
from nodes import CLIPSetLastLayer
clip_set_last_layer = CLIPSetLastLayer()
clip = clip_set_last_layer.set_last_layer(stop_at_clip_layer=2)

In [ ]:
from comfy_extras.nodes_clip_sdxl import CLIPTextEncodeSDXL
clip_text_encode = CLIPTextEncodeSDXL()
encode_p = clip_text_encode.encode(clip=clip, target_width=width, target_height=height, text_g=response, text_l=response ,width=4096, height=4096, crop_w=0, crop_h=0)

encode_n = clip_text_encode.encode(clip=clip, target_width=width, target_height=height, text_g="(deformed, distorted, disfigured:1.3), poor architecture, bad architecture, poorly drawn architecture, text, watermark, logo, bad proportions, duplicate, multiple angles, double image, two buildings, extra windows, missing windows, wrong perspective, tilted horizon, curved lines, bent walls, asymmetrical:1.2, low quality, worst quality, jpeg artifacts, blurry, out of focus, noise, grainy, oversaturated, cartoonish, 3d rendering artifacts, CGI artifacts, unrealistic shadows:1.3, incorrect shadows, multiple shadows, floating elements, disconnected elements, (incomplete walls, broken walls:1.2), incorrect reflections, bad composition, amateur, oversimplified, plastic look, fireplace, nude, women, men", text_l="(deformed, distorted, disfigured:1.3), poor architecture, bad architecture, poorly drawn architecture, text, watermark, logo, bad proportions, duplicate, multiple angles, double image, two buildings, extra windows, missing windows, wrong perspective, tilted horizon, curved lines, bent walls, asymmetrical:1.2, low quality, worst quality, jpeg artifacts, blurry, out of focus, noise, grainy, oversaturated, cartoonish, 3d rendering artifacts, CGI artifacts, unrealistic shadows:1.3, incorrect shadows, multiple shadows, floating elements, disconnected elements, (incomplete walls, broken walls:1.2), incorrect reflections, bad composition, amateur, oversimplified, plastic look, fireplace, nude, women, men" ,width=4096, height=4096, crop_w=0, crop_h=0)


In [ ]:
from custom_nodes.advancedControlNet.adv_control.nodes import AdvancedControlNetApplySingle
advanced_controlnet_apply = AdvancedControlNetApplySingle()
control_positive, control_negative = advanced_controlnet_apply.apply_controlnet(positive=encode_p, negative=encode_n, control_net=controlnet, image=aio_image, strength=0.90, start_percent=0.0, end_percent=0.1, vae_optional=vae, model_opt=model)


In [ ]:
from custom_nodes.ipadapter.IPAdapterPlus import IPAdapterUnifiedLoader
ipadapter_unified_loader = IPAdapterUnifiedLoader()
model, ipadapter = ipadapter_unified_loader.load_models(model=model, preset="PLUS (high strength)")

In [ ]:
from nodes import CLIPVisionLoader
clip_vision_loader = CLIPVisionLoader()
clip_vision, = clip_vision_loader.load_clip(clip_name="CLIP-ViT-H-14-laion2B-s32B-b79K.bin")

In [ ]:
from custom_nodes.ipadapter.IPAdapterPlus import IPAdapterAdvanced
ipadapter_advanced = IPAdapterAdvanced()
ipadap_model, = ipadapter_advanced.apply_ipadapter(model=model, ipadapter=ipadapter, image=image, clip_vision=clip_vision, weight=0.65, weight_type="composition", combine_embeds= "concat", start_at=0.1, end_at=0.7, embeds_scaling="V only")

In [ ]:
from comfy_extras.nodes_pag import PerturbedAttentionGuidance
perturbed_attention_guidance = PerturbedAttentionGuidance()
pag_model, = perturbed_attention_guidance.patch(model=ipadap_model, scale=5)

In [ ]:
from custom_nodes.efficientNodes.efficiency_nodes import TSC_KSampler
tsc_ksampler = TSC_KSampler()
final_model, conditioning, conditioning_negative, latent, vae, image = tsc_ksampler.sample(model=pag_model, seed=-1, steps=18, cfg=5.0, sampler_name="euler_ancestral", scheduler="AYS SDXL", positive=control_positive, negative= control_negative, latent_image=latent, denoise=1.0, preview_method="auto", vae_decode="true", script=None, optional_vae=vae, vae_decode="true")

In [ ]:
from comfy_extras.nodes_upscale_model import UpscaleModelLoader
upscale_model_loader = UpscaleModelLoader()
upscale_model, = upscale_model_loader.load_model(model_name="4x-UltraSharp.pth")

In [ ]:
from comfy_extras.nodes_upscale_model import ImageUpscaleWithModel
image_upscale_with_model = ImageUpscaleWithModel()
final_image, = image_upscale_with_model.upscale(upscale_model=upscale_model, image=image)